# Combinations vectorized

#### Uses the following information:
- parents of pairs
- pairs indices
- Usual starts,stops,counts etc.

#### Basic idea:

We can note that `left` is the quotient of division between `pairs_index-pairs_start[event]` and `counts[event]`. `right` on the other hand, is the result of modulus of the abovementioned factors. We can use it to form the result.


In [1]:
import numpy as np
import sys
import numba

In [2]:
NUMEVENTS = 50
AVENUMJETS = 10

numjets = np.random.poisson(AVENUMJETS, NUMEVENTS).astype(np.int)
stops = np.cumsum(numjets).astype(np.int)
starts = np.zeros_like(stops)
starts[1:] = stops[:-1]

counts = stops-starts
offsets = np.zeros(len(numjets)+1)
offsets[1:] = stops[:]

In [3]:
parents = np.empty(stops[-1], dtype=np.int)

@numba.jit()
def vectorized_search(offsets, content):
    index = np.arange(len(content), dtype=int)                     # threadIdx.x on CUDA
    below = np.zeros(len(content), dtype=int)                      # just below = 0 on CUDA
    above = np.ones(len(content), dtype=int) * (len(offsets) - 1)  # same for above
    while True:
        middle = (below + above) // 2

        change_below = offsets[middle + 1] <= index                   # which "belows" must we change?
        change_above = offsets[middle] > index                        # which "aboves"?

        if not np.bitwise_or(change_below, change_above).any():    # neither? great! we're done!
            break
        else:
            below = np.where(change_below, middle + 1, below)      # vectorized "if" statement
            above = np.where(change_above, middle - 1, above)      # this is the only branch

    return middle

done!


In [48]:
# pairs_indices should properly be called pairs_counts
pairs_indices = np.zeros(len(numjets)+1)
pairs_indices[1:] = np.cumsum(counts*counts)
pairs_indices = pairs_indices.astype(np.int)

In [14]:
# pairs_contents should be called pairs_indices
pairs_contents = np.arange(pairs_indices[-1]).astype(np.int)
pairs_parents = vectorized_search(pairs_indices, pairs_contents)
pairs_parents = pairs_parents.astype(np.int)

done!


In [18]:
left = np.empty_like(pairs_contents)
right = np.empty_like(pairs_contents)

In [84]:
left[pairs_contents] = starts[pairs_parents[pairs_contents]] + np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/counts[pairs_parents[pairs_contents]]).astype(np.int)
right[pairs_contents] = starts[pairs_parents[pairs_contents]]+(pairs_contents-pairs_indices[pairs_parents[pairs_contents]])-counts[pairs_parents[pairs_contents]]*np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/counts[pairs_parents[pairs_contents]])

In [88]:
for i in range(6):
    print("Event {}\n Left {}\n \nRight {}\n \n".format(i,left[pairs_indices[i]:pairs_indices[i+1]], right[pairs_indices[i]:pairs_indices[i+1]]))

Event 0
 Left [0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3 3 3 3 3 3 4 4 4 4 4 4 4 5 5
 5 5 5 5 5 6 6 6 6 6 6 6]
 
Right [0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1
 2 3 4 5 6 0 1 2 3 4 5 6]
 

Event 1
 Left [ 7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9
  9  9  9  9  9 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14
 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16 16 16]
 
Right [ 7  8  9 10 11 12 13 14 15 16  7  8  9 10 11 12 13 14 15 16  7  8  9 10 11
 12 13 14 15 16  7  8  9 10 11 12 13 14 15 16  7  8  9 10 11 12 13 14 15 16
  7  8  9 10 11 12 13 14 15 16  7  8  9 10 11 12 13 14 15 16  7  8  9 10 11
 12 13 14 15 16  7  8  9 10 11 12 13 14 15 16  7  8  9 10 11 12 13 14 15 16]
 

Event 2
 Left [17 17 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 19 19 19 19 19
 19 19 19 19 19 20 20 20 20 20 20 20 20 20 20 21